In [50]:
from io import BytesIO

import requests
import pandas as pd

from decimal import Decimal

from yandex_geocoder import Client

import folium


In [29]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1iSrqedH09Ck2bTiTIo57whu94daXHMls6ig-748UC5s/edit#gid=0'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

r = requests.get(csv_export_url)
data = r.content

In [30]:
df = pd.read_csv(BytesIO(data))

,name,address
0,МГТУ им. Баумана,"2-я Бауманская ул., 5, стр. 4, Москва"
1,ГБОУ № 444,"Нижний Журавлёв пер., 3, Москва"
2,Group-IB,"Шарикоподшипниковская ул., 1, Москва"
3,"Оздоровительный центр ""Полюс""","Россия, Московская область, городской округ По..."


In [5]:
# geolocator = Nominatim(user_agent="freedom-sach-map")

In [37]:
api_key = '4daecbaf-37cc-47f6-9602-55e281126741'
client = Client(api_key)

In [43]:
lat = []
long = []

for i in df['address'].to_list():
    coordinates = client.coordinates(i)
    long.append(str(coordinates[0]))
    lat.append(str(coordinates[1]))

In [46]:
df['lat'] = lat
df['long'] = long

In [136]:
moscow = [55.60, 37.618]

m1 = folium.Map(location=moscow,
                # tiles='openstreetmap',
                tiles='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png',
                attr='dark_all''&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>, &copy; <a href="https://carto.com/attributions">CARTO</a>',
                zoom_start=9)

m1

In [137]:
df

,name,address,lat,long
0,МГТУ им. Баумана,"2-я Бауманская ул., 5, стр. 4, Москва",55.765772,37.68562
1,ГБОУ № 444,"Нижний Журавлёв пер., 3, Москва",55.78359,37.706164
2,Group-IB,"Шарикоподшипниковская ул., 1, Москва",55.722953,37.66565
3,"Оздоровительный центр ""Полюс""","Россия, Московская область, городской округ По...",55.32749,37.696319


In [138]:
for i in sorted(df['name'].unique()):
    temp = df[df['name']==i]
    
    lat = temp['lat'].astype(float).to_list()[0]
    long = temp['long'].astype(float).to_list()[0]
    
    folium.Marker(
        location=[lat, long],
        # popup='<i>Здесь был Вася</i>',
        tooltip=f'{i}').add_to(m1)

In [139]:
m1

In [124]:
m1.save('docs/index.html')